# Ensemple CB recommender based on Sparse Autoencoder

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../lib')

import torch
import numpy as np
import pandas as pd

import pytorch_common.util as pu

import logging
import data as dt

import recommender as rc

<Figure size 432x288 with 0 Axes>

## Setup

In [3]:
pu.LoggerBuilder().on_console().build()

<RootLogger root (INFO)>

In [4]:
pu.set_device_name('gpu')

In [5]:
pu.get_device()

device(type='cuda', index=0)

In [6]:
torch.cuda.is_available()

True

In [7]:
torch.__version__

'1.11.0'

## Evaluación

In [8]:
def load(name): return pd.read_json(f'../datasets/movie_{name}_embedding.json')
def emb_rec(emb, name): return rc.DistanceMatrixRecommender(emb, f'{name}_embedding')

In [9]:
embeddings_by_name   = {name: load(name) for name in ['genres', 'overview', 'tags']}
recommenders_by_name = {name: emb_rec(emb, name) for name, emb in embeddings_by_name.items()}
emb_recommenders     = list(recommenders_by_name.values())

Building Distances Matrix:   0%|          | 0/18515 [00:00<?, ?it/s]

Building Distances Matrix:   0%|          | 0/18515 [00:00<?, ?it/s]

Building Distances Matrix:   0%|          | 0/18515 [00:00<?, ?it/s]

In [10]:
item_index = 10

In [11]:
recommender_group = rc.RecommenderGroup(emb_recommenders)

In [12]:
recommender_group.recommend(item_index).show()


Recommender: genres
Item


,id,title
10,8360,Shrek 2


Recommendations


,index,distance,id,title,genres
0,10,0.000000,8360,Shrek 2,Adventure Animation Children Comedy Musical Ro...
1,2858,0.000323,1907,Mulan,Adventure Animation Children Comedy Drama Musi...
2,3208,0.001238,3159,Fantasia 2000,Animation Children Musical IMAX
3,1411,0.002851,3396,"Muppet Movie, The",Adventure Children Comedy Musical
4,2642,0.002851,107,Muppet Treasure Island,Adventure Children Comedy Musical



Recommender: overview
Item


...

,index,distance,id,title,overview
0,10,0.000000,8360,Shrek 2,"Shrek, Fiona and Donkey set off to Far, Far Aw..."
1,1173,0.014001,1663,Stripes,"John Winger, an indolent sad sack in his 30s, ..."
2,5521,0.014180,5366,Whore,This melodrama investigates the life of a pros...
3,11700,0.016830,4567,American Ninja 3: Blood Hunt,"Jackson is back, and now he has a new partner,..."
4,6380,0.017629,91483,Bullet to the Head,"After watching their respective partners die, ..."



Recommender: tags
Item


,id,title
10,8360,Shrek 2


Recommendations


,index,distance,id,title,tags
0,10,0.000000,8360,Shrek 2,(s)vcd 2020-2 relook 4 andrew adamson animatio...
1,124,0.017794,8784,Garden State,acting actors adventure annoying answering mac...
2,2416,0.018910,158813,Alice Through the Looking Glass,3d acting adventure alice in wonderland altern...
3,1830,0.020201,31696,Constantine,action adapted from:comic aftercreditsstinger ...
4,2266,0.020488,114552,"Boxtrolls, The",2015 academy award nomination animated feature...


In [29]:
recommender = rc.EnsembleWeightedDistanceMatrixRecommender(
    emb_recommenders,
    weights = { 'tags': 0.25, 'overview': 0.25, 'genres': 0.4}
)

In [30]:
recommender.recommend(item_index, k=5).show()


Recommender: ['genres', 'overview', 'tags']
Item


,id,title
10,8360,Shrek 2


Recommendations


,distance,title,genres,overview,tags
0,0.000000,Shrek 2,Adventure Animation Children Comedy Musical Ro...,"Shrek, Fiona and Donkey set off to Far, Far Aw...",(s)vcd 2020-2 relook 4 andrew adamson animatio...
1,0.011675,Peter Pan,Action Adventure Children Fantasy,"In stifling Edwardian London, Wendy Darling me...",adapted from:book animal:crocodile author:j. m...
2,0.011875,"Secret of NIMH, The",Adventure Animation Children Drama,A widowed field mouse must move her family -- ...,2d animation adapted from:book animated animat...
3,0.016953,Creature Comforts,Animation Comedy,Interviews with the public are used to put wor...,aardman aardman studios animal movie animation...
4,0.019066,Flushed Away,Animation Comedy,"London high-society mouse, Roddy is flushed do...",31-3-2007 aardman adventure andy serkis animat...
